In [8]:
import string
import numpy as np
from collections import Counter
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

def load_file(filename):
    with open('transcripts.txt',"r") as transcript_file:
        transcript = transcript_file.read()
    transcript_file.close()
    return transcript

def clean_doc(file):
    file = file.replace("\n,", "[").replace('",,' , "\n\n\n\n").replace(":" , "]:").replace("[[", "[").replace("," , "").replace("  " , " ").replace("\n[\n", "").replace("\t", " ").replace("'", "")
    file = file.replace('"', "")
    tokens = file.split()
    tokens = [word for word in tokens if word.isalpha()]
    tokens = [word.lower() for word in tokens]
    return file

def punct_replace(file):
    file = file.replace('\n', " NEWLINE ").replace( '!', " EXCLAMATIONMARK").replace("$", "DOLLARSIGN ")
    file = file.replace(".", " FULLSTOP ").replace("/", " FORWARDSLASH ").replace(":", " COLON")
    file = file.replace("?", " QUESTIONMARK").replace("[", "LEFTSQUAREBRACKET ").replace("]", " RIGHTSQUAREBRACKET")
    return file


in_filename = 'transcripts.txt'
in_file = load_file(in_filename)

cleaned_file = clean_doc(in_file)

punct_replaced = punct_replace(cleaned_file)
punct_replaced = punct_replaced.lower()
words_in_text = punct_replaced.split()



#corpus = sorted(list(set(clean_file)))
#print(corpus)

#print('Dataset Stats')
#print('Roughly the number of unique words: {}'.format(len({word: None for word in cleaned_file.split()})))
scenes = cleaned_file.split('\n\n\n\n\n\n')
print('Number of episodes: {}'.format(len(scenes)))
#sentence_count_scene = [scene.count('\n') for scene in scenes]
#print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

#sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
#print('Number of lines: {}'.format(len(sentences)))
#word_count_sentence = [len(sentence.split()) for sentence in sentences]
#print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

#print()
#print('The sentences {} to {}:'.format(*view_sentence_range))
#print('\n'.join(cleaned_file.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))


Number of episodes: 418


In [11]:
    word_freq = {}
    for word in words_in_text:
        word_freq[word] = word_freq.get(word, 0) + 1

    ignored_words = set()
    for k, v in word_freq.items():
        if word_freq[k] < 10:
            ignored_words.add(k)
    words = set(words_in_text)
    print('Unique words before ignoring:', len(words))
    print('Ignoring words with frequency <', 10)
    words = sorted(set(words) - ignored_words)
    print('Unique words after ignoring:', len(words))

    word_indices = dict((c, i) for i, c in enumerate(words))
    indices_word = dict((i, c) for i, c in enumerate(words))

    # cut the text in semi-redundant sequences of SEQUENCE_LEN words
    sentences = []
    next_words = []
    ignored = 0
    SEQUENCE_LENGTH = 16

    for i in range(0, len(words_in_text) - SEQUENCE_LENGTH, 1):
        # Only add the sequences where no word is in ignored_words
        if len(set(words_in_text[i: i+SEQUENCE_LENGTH+1]).intersection(ignored_words)) == 0:
            sentences.append(words_in_text[i: i + SEQUENCE_LENGTH])
            next_words.append(words_in_text[i + SEQUENCE_LENGTH])
        else:
            ignored = ignored + 1
    print('Ignored sequences:', ignored)
print('Remaining sequences:', len(sentences))
print('Number of unique words(corpus size):',len(words))

Unique words before ignoring: 21091
Ignoring words with frequency < 10
Unique words after ignoring: 4970
Ignored sequences: 467747
Remaining sequences: 770290
Number of unique words(corpus size) 4970


In [3]:
if 'yagh' in next_words:
    print('yes')
else:
    print('no')

no


In [35]:
def shuffle_and_split_training_set(sentences_original, next_original, percentage_test=2):
    # shuffle at unison
    print('Shuffling sentences')

    tmp_sentences = []
    tmp_next_word = []
    for i in np.random.permutation(len(sentences_original)):
        tmp_sentences.append(sentences_original[i])
        tmp_next_word.append(next_original[i])

    cut_index = int(len(sentences_original) * (1.-(percentage_test/100.)))
    x_train, x_test = tmp_sentences[:cut_index], tmp_sentences[cut_index:]
    y_train, y_test = tmp_next_word[:cut_index], tmp_next_word[cut_index:]

    print("Size of training set = %d" % len(x_train))
    print("Size of test set = %d" % len(y_test))
    return (x_train, y_train), (x_test, y_test)

(sentences, next_words), (sentences_test, next_words_test) = shuffle_and_split_training_set(sentences, next_words)

Shuffling sentences
Size of training set = 754884
Size of test set = 15406


In [34]:
print((3 ))

3


In [36]:
from keras.models import Sequential
from keras import losses
from keras.layers import Dense, Activation, LSTM, Bidirectional, Dropout, Embedding
from keras.callbacks import LambdaCallback, ModelCheckpoint, EarlyStopping
from keras.optimizers import RMSprop
import keras
import os
import sys

def get_model(dropout=0.2):
    model = Sequential()
    model.add(Embedding(input_dim=len(words), output_dim=1024))
    model.add(Bidirectional(LSTM(128)))
    if dropout > 0:
        model.add(Dropout(dropout))
    model.add(Dense(len(words)))
    model.add(Activation('softmax'))
    return model
    

def generator(sentences, next_words, batch_size):
    index = 0
    while True:
        x = np.zeros((batch_size, 16), dtype=np.int32)
        y = np.zeros((batch_size), dtype=np.int32)
        for i in range(batch_size): 
            for num, word in enumerate(sentences[index % len(sentences)]):
                x[i, num] = word_indices[word]
            y[i] = word_indices[next_words[index % len(sentences)]]
            index = index + 1
        yield x, y
model = get_model()
model.summary()
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 1024)        5089280   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               1180672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 4970)              1277290   
_________________________________________________________________
activation_1 (Activation)    (None, 4970)              0         
Total params: 7,547,242
Trainable params: 7,547,242
Non-trainable params: 0
___________

In [11]:
def on_epoch_end(epoch, logs):
    in_filename = 'predicted.txt'
    f=open("predicted.txt","a")
    # Randomly pick a seed sequence
    seed_index = np.random.randint(len(sentences+sentences_test))
    seed = (sentences+sentences_test)[seed_index]

    for diversity in [0.3, 0.4, 0.5, 0.6, 0.7]:
    #for diversity in [0.7]:
        sentence = seed
        f.write('%02d %.3f\n' % (epoch, logs['loss']))
        f.write('\n ----- Diversity:' + str(diversity) + '\n')
        f.write('----- Generating with seed:\n"' + ' '.join(sentence)+ '\n\n')
        predicted_text = ""
        for i in range(640):
            x_pred = np.zeros((1, 16))
            for t, word in enumerate(sentence):
                x_pred[0, t] = word_indices[word]
            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_word = indices_word[next_index]
            sentence = sentence[1:]
            sentence.append(next_word)
            predicted_text += (next_word + " ")
        f.write(predicted_text)
        f.write('='*80 + '\n')
        file = load_file(in_filename)
        cleaned = post_process(file) 
        w=open("cleanpredicted.txt","w+")
        w.write(cleaned)
        
        
        
        
def load_file(filename):
    with open('predicted.txt',"r") as transcript_file:
        transcript = transcript_file.read()
    transcript_file.close()
    return transcript



def post_process(file):
    file = file.replace("newline", '\n').replace("exclamationmark", '!').replace("dollarsign ","$")
    file = file.replace(" fullstop",".").replace("forwardslash ","/").replace("colon",":")
    file = file.replace(" questionmark","?").replace("leftsquarebracket ","[").replace(" rightsquarebracket","]")
    return file




In [12]:


model.compile(loss='sparse_categorical_crossentropy', optimizer='adam' , metrics=['accuracy'])

#model.fit_generator(generator(sentences, next_words, BATCH_SIZE), 
 #                   steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1, 
  #                                      epochs=10, 
   #                                    validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
    #                                   validation_steps=int(len(sentences_test)/BATCH_SIZE)+1)

In [15]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)
early_stopping = EarlyStopping(monitor='val_acc', patience=20)
callbacks_list = [print_callback, early_stopping]
BATCH_SIZE= 32

In [ ]:
   model.fit_generator(generator(sentences, next_words, BATCH_SIZE),
                        steps_per_epoch=int(len(sentences)/BATCH_SIZE) + 1,
                        epochs=100,
                        callbacks=callbacks_list,
                        validation_data=generator(sentences_test, next_words_test, BATCH_SIZE),
                        validation_steps=int(len(sentences_test)/BATCH_SIZE) + 1)

Epoch 1/100
10417/10417 [==============================] - 328s 31ms/step - loss: 3.8092 - acc: 0.3357 - val_loss: 3.6986 - val_acc: 0.3417
Epoch 2/100
10417/10417 [==============================] - 329s 32ms/step - loss: 3.4945 - acc: 0.3593 - val_loss: 3.6114 - val_acc: 0.3512
Epoch 3/100
10417/10417 [==============================] - 328s 31ms/step - loss: 3.3153 - acc: 0.3736 - val_loss: 3.5974 - val_acc: 0.3587


/scratch/pygpu_labs/envs/Labs_y3/lib/python3.6/site-packages/ipykernel_launcher.py:37: RuntimeWarning: divide by zero encountered in log


Epoch 4/100
10417/10417 [==============================] - 327s 31ms/step - loss: 3.1763 - acc: 0.3855 - val_loss: 3.6064 - val_acc: 0.3611
Epoch 5/100
10417/10417 [==============================] - 327s 31ms/step - loss: 3.0631 - acc: 0.3959 - val_loss: 3.6404 - val_acc: 0.3556
Epoch 6/100
10417/10417 [==============================] - 329s 32ms/step - loss: 2.9693 - acc: 0.4050 - val_loss: 3.6641 - val_acc: 0.3553
Epoch 7/100
10417/10417 [==============================] - 328s 32ms/step - loss: 2.8909 - acc: 0.4127 - val_loss: 3.7045 - val_acc: 0.3552
Epoch 8/100
 1051/10417 [==>...........................] - ETA: 4:52 - loss: 2.8391 - acc: 0.4195